In [1]:
import zipfile
import numpy as np
import pandas
from pprint import pprint

In [2]:
zip_ref = zipfile.ZipFile('1927eve.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

zip_ref = zipfile.ZipFile('1938eve.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()


In [86]:
filename = '1927DET.EVA'
gamelogs = []
gamecount = -1

for line in open(filename).readlines():
    linearray = line.split(',')
#    print(linearray)
    if (linearray[0] == 'id'):
        # 'id' starts the next game, so immediately advance gamecount  
        gamecount = gamecount + 1
        
        # record prior game data by adding it to the gamelogs list, unless gamecount = -1
        if (gamecount > 0):
            thisgamedict['info'] = info_dict
            thisgamedict['visiting_starters'] = vis_start_list
            thisgamedict['home_starters'] = home_start_list

            if (len(vis_sub_list) > 0):
                thisgamedict['visiting_subs'] = vis_sub_list
            if (len(home_sub_list) > 0):
                thisgamedict['home_subs'] = home_sub_list            

        gamelogs.append(thisgamedict)
        
        # initialize new game
        thisgamedict = {}
        info_dict = {}
        start_dict = {}
        vis_start_list = np.empty((9,4),dtype=object)
        home_start_list = np.empty((9,4),dtype=object)
        vis_sub_list = np.empty((0,4),dtype=object)
        home_sub_list = np.empty((0,4),dtype=object)
        plays = np.empty((0,6),dtype=object)
        data_dict = {}
        
        thisgamedict['id'] = linearray[1] 
#        if (gamecount > 0):
#            print(thisgamedict['id'],' had {:.0f} batters.'.format(batterscount))
        batterscount = 0
    if (linearray[0] == 'version'):
        thisgamedict['version'] = linearray[1]
    if (linearray[0] == 'info'):
        info_dict[linearray[1]] = linearray[2]
    if (linearray[0] == 'start'):
        batterscount = batterscount + 1
        if (linearray[3] == '0'):
            vis_start_list[batterscount-1,0] = linearray[1]
            vis_start_list[batterscount-1,1] = linearray[2]
            vis_start_list[batterscount-1,2] = linearray[4]
            vis_start_list[batterscount-1,3] = linearray[5]
        elif (linearray[3] == '1'):
            home_start_list[batterscount-10,0] = linearray[1]
            home_start_list[batterscount-10,1] = linearray[2]
            home_start_list[batterscount-10,2] = linearray[4]
            home_start_list[batterscount-10,3] = linearray[5]            
        else:
            print('NEITHER VISITOR NOR HOME????')

    if (linearray[0] == 'sub'):
        this_sub = []
        this_sub.append(linearray[1])
        this_sub.append(linearray[2])
        this_sub.append(linearray[4])
        this_sub.append(linearray[5])
        if (linearray[3] == '0'):
            vis_sub_list = np.vstack((vis_sub_list,this_sub))
        elif (linearray[3] == '1'):
            home_sub_list = np.vstack((home_sub_list,this_sub))
        else:
            print('NEITHER VISITOR NOR HOME????')
            
    if (linearray[0] == 'play'):
#        print(linearray)
        thisplay = []
        for i in range(1,7):
            thisplay.append(linearray[i])
        plays = np.vstack((plays,thisplay))
#        plays
pprint(gamelogs)
#print(plays)
print('ahoy')

[{'id': 'DET192710022\n', 'version': '1\n'},
 {'home_starters': array([['tavej101', '"Jackie Tavener"', '1', '6\n'],
       ['mcmam101', '"Marty McManus"', '2', '4\n'],
       ['manuh101', '"Heinie Manush"', '3', '8\n'],
       ['fothb101', '"Bob Fothergill"', '4', '7\n'],
       ['heilh101', '"Harry Heilmann"', '5', '9\n'],
       ['bluel101', '"Lu Blue"', '6', '3\n'],
       ['warnj101', '"Jack Warner"', '7', '5\n'],
       ['sheam102', '"Merv Shea"', '8', '2\n'],
       ['hollk101', '"Ken Holloway"', '9', '1\n']], dtype=object),
  'home_subs': array([['neunj101', '"Johnny Neun"', '8', '11\n'],
       ['winga101', '"Al Wingo"', '9', '11\n'],
       ['woodl101', '"Larry Woodall"', '8', '2\n'],
       ['white102', '"Earl Whitehill"', '9', '1\n']], dtype=object),
  'id': 'DET192704280\n',
  'info': {'attendance': '0\n',
           'date': '1927/04/28\n',
           'daynight': 'day\n',
           'fieldcond': 'unknown\n',
           'gwrbi': '\n',
           'hometeam': 'DET\n',
       